In [4]:
import autoreload

%load_ext autoreload
%autoreload 2

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim
import time
import os
from collections import OrderedDict
from torch.utils.data import Subset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,download=True, transform=transform)
#trainloader = torch.utils.data.DataLoader(trainset, batch_size=16,
#                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

#testloader = torch.utils.data.DataLoader(testset, batch_size=16,
#                                         shuffle=False, num_workers=2)

classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

100.1%

Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw


113.5%

Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw


100.4%

Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw


180.4%

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw
Processing...
Done!


In [7]:
def trainset_node_split(dataset, N, seed = 0):
    np.random.seed(seed)
    a = np.arange(len(dataset))
    np.random.shuffle(a)
    datasets = {}
    size = int(len(dataset)/N)
    for i in range(N):
        datasets[i] = Subset(dataset, a[i*size:(i+1)*size].tolist())
    return datasets

In [8]:
m = trainset_node_split(trainset, 2)

In [10]:
class Node():
    """Node(Choco_Gossip): x_i(t+1) = x_i(t) + gamma*Sum(w_ij*[xhat_j(t+1) - xhat_i(t+1)])"""
    
    def __init__(self, gamma, loader, model, criterion):
        
        self.neighbors = []
        self.neighbor_wts = {}
        
        self.step_size = gamma
                
        self.dataloader = loader
        
        self.model = 
        
        self.x_i = OrderedDict()
        
        self.model_params = []
        
        for (k,v) in self.model.net.state_dict().items():
            
            self.model_params.append(k)
            self.x_i[k] = v.clone().detach()
            
        #for a in self.model.parameters():
        #    self.x_i.append(a)
        
        self.criterion = params["criterion"]
        
        self.dataiter = iter(dataloader)
        
    
    def compute_gradient(self, quantizer, optimizer):
        """Computes nabla(x_i, samples) and returns estimate after quantization"""
        
        # Sample batch from loader #
        
        inputs, targets = self.dataiter.next()
        
        outputs = self.model(inputs)
                
        loss = self.criterion(outputs, targets)
        
        #optimizer.zero_grad()
        
        loss.backward()
        
        gt = OrderedDict()
        
        for (k,v) in self.model.net.state_dict().items():
            
            gt[k] = quantizer(v.grad.clone().detach())
            
        #optimizer.step()
    
        self.curr_gt = gt
        
        return
    
    def assign_params(self, W):
        """Assign dict W to model"""
        
        with torch.no_grad():
            self.model.load_state_dict(W, strict=False)
        
        return
    
    def update_model(self, W):
        
        ### Implement Algorithm ###
        
        ## Assign Parameters after obtaining Consensus##
        
        assign_params(W)
        
        return        

In [ ]:
class Network():
    """Define graph"""
    
    def __init__(self, W, models, learning_rates, loaders, criterion):
        
        self.adjacency = W
        self.num_nodes = W.shape[0]
        
        self.nodes = OrderedDict()
        
        for i in range(num_nodes):
            nodes[str(i)] = Node(learning_rates[i], loaders[i],models[i], criterion)
            for j in range(num_nodes):
                if(j != i and W[i, j] > 0):
                    nodes[str(i)].neighbors.append(j)
                    nodes[str(i)].neighbor_wts[str(j)] = W[i, j]
                    
            
    def simulate(self, iterations, epochs):
        
        for i in range(epochs):
            for j in range(iterations):
                for k in range(num_nodes):
                    nodes[str(k)].compute_gradient()
                    
                for k in range(num_nodes):
                    nodes[str(k)].update_model()       